# Exemplary use cases for the Co2calculator
In this Jupyter notebook you will find some helpful examples of how to use the co2calculator.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
from dotenv import load_dotenv
import os

sys.path.insert(0, Path.cwd().parent.parent.absolute().as_posix())

import co2calculator as co2

from co2calculator.parameters import *

from co2calculator.api.energy import Energy
from co2calculator.api.trip import Trip

In [2]:
load_dotenv()

# Load environment vars
ORS_API_KEY = os.environ.get("ORS_API_KEY")

In [3]:
#print(ORS_API_KEY)

## Examples for the Energy class 

Let's take a look at some examples of the Energy class. It has two methods from_electricity and from_heating. First, you must initialize the Energy object with your consumption. Depending on the fuel type, it can take on different units. <br>
Then we can calculate the emissions for an electricity consumption of 100kWh in Germany.

In [4]:
# Initialize Energy object 
energy=Energy(consumption=100)

# calculate emissions for electricity 
electricity_result=energy.from_electricity(country_code="DE").calculate_co2e()

print(f"The result for electricity calculations is: {electricity_result}\n")

The result for electricity calculations is: EnergyEmissions(co2e=44.912, emission_factor=0.44912, emission_parameters=ElectricityEmissionParameters(category=<EmissionCategory.ELECTRICITY: 'electricity'>, fuel_type=<ElectricityFuel.PRODUCTION_FUEL_MIX: 'production fuel mix'>, country_code='DE', own_share=1.0), consumption=100)



You can use the same Energy class object to calculate the heating emissions. The default fuel type is gas. Now you only need to set in_kwh to True. Otherwise, you must specify a fuel type.

In [5]:
# calculate emissions for heating 
heating_result=energy.from_heating(in_kwh=True).calculate_co2e()

print(f"The emissions for the heating calculations with 100 kWh of gas are:{heating_result}")

TypeError: Energy.from_heating() got an unexpected keyword argument 'in_kwh'

If you want to know the respectively unit of each fuel type you can use the method get_options().

In [ ]:
# Initialize Energy object 
energy=Energy(consumption=100, fuel_type='oil')

# get options of heating
heating_options=energy.from_heating().get_options()

print(f"The options for heating are:{heating_options}")

# calculate emissions of oil
heating_oil=energy.from_heating().calculate_co2e()

print(f"The emissions for the heating calculations with 100 liter of oil are:{heating_oil}")

The user can also provide further information, e.g. the fuel type or how large their share of the energy consumed is.
Let's assume you use the normal production fuel mix and share your office with another person.

In [ ]:
# electricity calculations with user defined input
electricity_energy=Energy(consumption=100, fuel_type="production fuel mix", own_share=0.5)

electricity_result2=electricity_energy.from_electricity(country_code="DE").calculate_co2e()

print(electricity_result2)

The same applies to heating. Let's assume you want to calculate the emissions for 100 kg of coal and you share your apartment with 4 other people.

In [ ]:
# heating calculations with more user defined input
heating_energy=Energy(consumption=100, fuel_type="coal", own_share=0.25)

heating_result3=heating_energy.from_heating().calculate_co2e()

print(heating_result3)

## Examples for the Trip class 
You have two options for calculating the emissions for the Trip class. Either you specify the distance in kilometers when initializing the Trip class object, or you define the start and destination of your trip.
### Trip calculations with a given distance
Let's assume you want to make a long journey of 5000 km and want to know which type of transportation causes the lowest emissions.

In [ ]:
# Initiate Trip object
trip=Trip(distance=5000)

In [ ]:
# trip by train
train_trip=trip.by_train().calculate_co2e()
print(train_trip)

For your plane trip you also want to compare the emissions when flying first class versus economy class.

In [ ]:
# first_class trip by plane
plane_first_class=trip.by_plane(seating="first_class").calculate_co2e()

print(plane_first_class)

In [ ]:
# economy_class trip by plane
plane_economy_class=trip.by_plane(seating="economy_class").calculate_co2e()

print(plane_economy_class)

Your last option is to take your big trip in an electric car, and you're traveling with two of your colleagues in a large car.

In [ ]:
# trip with electric car
car=trip.by_car(fuel_type="electric", size="large", passengers=3).calculate_co2e()
print(car)

### Provide your own emission factor
If you know for example the emission factor of your car, you can use the method by_custom() to calculate the emissions of your trip.

In [ ]:
custom_emission_factor = trip.by_custom(emission_factor=0.1).calculate_co2e()
print(custom_emission_factor)

### Trip calculations with input start and destination
Enter the start and destination either as a dictionary or as a string.

In [15]:
# provide start and destination as dictionary
start = {"locality": "Heidelberg", "country": "Germany", "address_type": "address",}
destination = {"locality": "Berlin", "country": "Germany", "address_type": "address",}

# initiate car Trip object with given start and destination
car_trip = Trip(start=start, destination=destination)

If you only want to know the distance, you can use the calculate_distance() method.

In [17]:
# calculate distance of trip by car --> remove?
car_distance = car_trip.by_car().calculate_distance()

4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]


Calculate the emissions of your car journey with calculate_co2e(), this also returns the locations used and all parameters used.

In [18]:
# calculate emissions of trip by car
car_co2e = car_trip.by_car().calculate_co2e()
print(car_co2e)

No address type provided: ('address', 'trainstation' ,'airport'), assume address
No address type provided: ('address', 'trainstation' ,'airport'), assume address
4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]
TransportEmissions(co2e=119.50935344, emission_factor=0.1864, emission_parameters=CarEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.CAR: 'car'>, fuel_type=<CarFuel.AVERAGE: 'average'>, size=<Size.AVERAGE: 'average'>, passengers=1), distance=641.1446)


If you want to travel by train, your start and destination could look like this.

In [19]:
# give start and destination as dictionary for train
start = {"station_name": "Heidelberg Hbf", "country": "DE", "address_type": "trainstation",}
destination = {"station_name": "Berlin Hbf", "country": "DE", "address_type": "trainstation",}

# initiate train Trip object with given start and destination
train_trip = Trip(start=start, destination=destination)

# calculate emissions of trip by train
train_co2e = train_trip.by_train().calculate_co2e()
print(train_co2e)

TransportEmissions(co2e=5.096385196187217, emission_factor=0.0068, emission_parameters=TrainEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.TRAIN: 'train'>, vehicle_range=<BusTrainRange.LONG_DISTANCE: 'long-distance'>, country_code='global'), distance=749.4684112040026)


In [22]:
plane= car_trip.by_plane().calculate_co2e()
print(plane)

No address type provided: ('address', 'trainstation' ,'airport'), assume address
No address type provided: ('address', 'trainstation' ,'airport'), assume address
4 location(s) found. Using this result: Heidelberg, Germany (data type: locality)
Coords:  [8.769869, 49.37625]
7 location(s) found. Using this result: Berlin, Germany (data type: locality)
Coords:  [13.407032, 52.524932]
TransportEmissions(co2e=182.71374403439816, emission_factor=0.3192, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.AVERAGE: 'average'>, vehicle_range=<FlightRange.SHORT_HAUL: 'short-haul'>), distance=572.4114788045056)


Travelling by plane your code could look like this. 

In [24]:
# start and destination for a trip by plane
start = {"IATA": "FRA", "address_type": "airport",}
destination = {"IATA": "BER", "address_type": "airport",}

# initiate plane Trip object with given start and destination
plane_trip = Trip(start=start, destination=destination)

# calculate emissions for trip by plane
plane_co2e = plane_trip.by_plane().calculate_co2e()
print(plane_co2e)

TransportEmissions(co2e=167.82950702986088, emission_factor=0.3192, emission_parameters=PlaneEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.PLANE: 'plane'>, seating=<FlightClass.AVERAGE: 'average'>, vehicle_range=<FlightRange.SHORT_HAUL: 'short-haul'>), distance=525.7816636273838)


Lets take the ferry.

In [26]:
# vergleich für ferry distanz


# provide start and destination as dictionary
start = {"locality": "Hamburg", "country": "Germany", "address_type": "address",}
destination = {"locality": "Cuxhaven", "country": "Germany", "address_type": "address",}

# initiate car Trip object with given start and destination
car_trip = Trip(start=start, destination=destination)

# calculate emissions of trip by car
car_co2e = car_trip.by_car().calculate_co2e()
print(car_co2e)

4 location(s) found. Using this result: Hamburg, Germany (data type: locality)
Coords:  [10.007046, 53.576158]
2 location(s) found. Using this result: Cuxhaven, Germany (data type: locality)
Coords:  [8.693474, 53.859527]
TransportEmissions(co2e=40.20196912, emission_factor=0.1864, emission_parameters=CarEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.CAR: 'car'>, fuel_type=<CarFuel.AVERAGE: 'average'>, size=<Size.AVERAGE: 'average'>, passengers=1), distance=215.67579999999998)


In [25]:
# provide start and destination
start = {"locality": "Hamburg", "country": "DE", "address_type": "address",}
destination = {"locality": "Cuxhaven", "country": "DE", "address_type": "address",}

# initiate ferry Trip object with given start and destination
ferry_trip = Trip(start=start, destination=destination)

# calculate emissions for ferry trip
ferry_co2e = ferry_trip.by_ferry().calculate_co2e()
print(ferry_co2e)

4 location(s) found. Using this result: Hamburg, Germany (data type: locality)
Coords:  [10.007046, 53.576158]
2 location(s) found. Using this result: Cuxhaven, Germany (data type: locality)
Coords:  [8.693474, 53.859527]
TransportEmissions(co2e=0.5411524139999999, emission_factor=0.11286, emission_parameters=FerryEmissionParameters(category=<EmissionCategory.TRANSPORT: 'transport'>, subcategory=<TransportationMode.FERRY: 'ferry'>, ferry_class=<FerryClass.AVERAGE: 'average'>), distance=4.794899999999999)
